# Building Blocks

> Building block assembly related functions

In [ ]:
#| default_exp building_blocks

In [ ]:
#| hide
from nbdev.showdoc import *
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
import chem_templates
from chem_templates.imports import *
from chem_templates.utils import *
from chem_templates.chem import Molecule
from rdkit import Chem

BB_PATH = chem_templates.__path__[0] + '/building_block_schemas'

/home/dmai/miniconda3/envs/chem_templates/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#| export
class BuildingBlockSchema():
    def __init__(self, 
                 name:              str, 
                 smarts_match_any:  list[str], 
                 smarts_match_all:  list[str], 
                 smarts_match_none: list[str]):
        
        self.name = name
        
        self.smarts_match_any = smarts_match_any
        self.smarts_match_any_mols = [Chem.MolFromSmarts(i) for i in smarts_match_any]
        
        self.smarts_match_all = smarts_match_all
        self.smarts_match_all_mols = [Chem.MolFromSmarts(i) for i in smarts_match_all]
        
        self.smarts_match_none = smarts_match_none
        self.smarts_match_none_mols = [Chem.MolFromSmarts(i) for i in smarts_match_none]
        
    def _check_match_any(self, mol: Chem.Mol) -> bool:
        for substruct in self.smarts_match_any_mols:
            if mol.HasSubstructMatch(substruct):
                return True
        return False
    
    def _check_match_all(self, mol: Chem.Mol) -> bool:
        for substruct in self.smarts_match_all_mols:
            if not mol.HasSubstructMatch(substruct):
                return False
        return True
    
    def _check_match_none(self, mol: Chem.Mol) -> bool:
        for substruct in self.smarts_match_none_mols:
            if mol.HasSubstructMatch(substruct):
                return False
        return True
        
    def match(self, mol: Chem.Mol) -> bool:
        return self._check_match_any(mol) and self._check_match_all(mol) and self._check_match_none(mol)
    
    @classmethod
    def from_dict(cls, input_dict: dict):
        '''
        input_dict = {
            'schema_name' : '',
            'smarts_match_any' : [...],
            'smarts_match_all' : [...],
            'smarts_match_none' : [...]
            }
        '''
        return cls(input_dict['schema_name'], 
                   input_dict['smarts_match_any'], 
                   input_dict['smarts_match_all'], 
                   input_dict['smarts_match_none'])
    
    @classmethod
    def from_file(cls, filename):
        with open(filename, 'r') as f:
            input_dict = json.load(f)
            return cls.from_dict(input_dict)
    
    def dump(self):
        return {
            'schema_name' : self.name,
            'smarts_match_any' : self.smarts_match_any,
            'smarts_match_all' : self.smarts_match_all,
            'smarts_match_none' : self.smarts_match_none
        }
    
    def __repr__(self) -> str:
        return self.name

In [ ]:
acid_schemas = json.load(open(f'{BB_PATH}/Acid.json'))
schema = BuildingBlockSchema.from_dict(acid_schemas['schemas'][0]) # aromatic acids

acid_mol = Chem.MolFromSmiles('[OH]C(=O)c1ccccc1')
non_acid_mol = Chem.MolFromSmiles('CC(=O)c1ccccc1')

assert schema.match(acid_mol)
assert not schema.match(non_acid_mol)

In [ ]:
#| export
class BuildingBlockClass():
    def __init__(self, 
                 name:    str, 
                 schemas: list[BuildingBlockSchema]):
        
        self.name = name
        self.schemas = schemas
        
    def match(self, mol: Chem.Mol) -> dict:
        output = {}
        for schema in self.schemas:
            if schema.match(mol):
                output[schema.name] = schema
                
        if output:
            output = {self.name : output}
            
        return output
    
    def add_schema(self, schema: BuildingBlockSchema):
        self.schemas.append(schema)
    
    @classmethod
    def from_dict(cls, input_dict: dict):
        '''
        input_dict = {
            'class_name' : '',
            'schemas' : [
                    {'schema_name' : '',
                    'smarts_match_any' : [...],
                    'smarts_match_all' : [...],
                    'smarts_match_none' : [...]},
                    ]
            }
        '''
        
        schemas = [BuildingBlockSchema.from_dict(i) for i in input_dict['schemas']]
        return cls(input_dict['class_name'], schemas)
    
    @classmethod
    def from_file(cls, filename):
        with open(filename, 'r') as f:
            input_dict = json.load(f)
            return cls.from_dict(input_dict)
    
    def dump(self):
        return {'class_name' : self.name, 'schemas' : [i.dump() for i in self.schemas]}
    
    def __repr__(self) -> str:
        return f"{self.name}: {', '.join([i.name for i in self.schemas])}"

In [ ]:
#| export

BB_CLASS_NAMES = [
        'SecondaryAmines',
        'Acylhalides',
        'SulfonesSulfinates',
        'Reagents',
        'Bifunctional',
        'ArylHalide',
        'Anhydrides',
        'Boronics',
        'Alkenes',
        'Acetylenes',
        'AlkylHalides',
        'Aldehyde',
        'Ketones',
        'Amides',
        'TertiaryAmines',
        'ElementOrganics',
        'SulfonylHalides',
        'PrimaryAmines',
        'Trifunctional',
        'Alcohols',
        'nHAzoles',
        'Azides',
        'Esters',
        'Aminoacids',
        'Acid',
        'ReagentsForOlefination'
]

BB_CLASSES = {i : BuildingBlockClass.from_file(f'{BB_PATH}/{i}.json') for i in BB_CLASS_NAMES}

In [ ]:
assert BB_CLASSES['ArylHalide'].match(Chem.MolFromSmiles('CC(C)(C)OC(=O)NNS(=O)(=O)c1ccc(F)cc1'))
assert not BB_CLASSES['SecondaryAmines'].match(Chem.MolFromSmiles('CC(C)(C)OC(=O)NNS(=O)(=O)c1ccc(F)cc1'))

In [ ]:
#| export
class ReactionUniverse():
    def __init__(self, 
                 universe_name: str, 
                 building_block_classes: list[BuildingBlockClass]):
        self.universe_name = universe_name
        self.building_block_classes = building_block_classes
        self.building_block_class_dict = {i.name : i for i in self.building_block_classes}
        
    def add_class(self, bb_class: BuildingBlockClass):
        self.building_block_classes.append(bb_class)
        self.building_block_class_dict[bb_class.name] = bb_class
        
    @classmethod
    def from_dict(cls, input_dict: dict):
        '''
        input_dict = {
            'universe_name' : '',
            'building_block_classes' : [
                {
                'class_name' : '',
                'schemas' : [
                        {'schema_name' : '',
                        'smarts_match_any' : [...],
                        'smarts_match_all' : [...],
                        'smarts_match_none' : [...]},
                        ]
                }
            ]
        }
        '''
        name = input_dict['universe_name']
        building_block_classes = [BuildingBlockClass.from_dict(i) for i in input_dict['building_block_classes']]
        return cls(name, building_block_classes)
        
    @classmethod
    def from_file(cls, filename):
        with open(filename, 'r') as f:
            input_dict = json.load(f)
            return cls.from_dict(input_dict)
        
    def dump(self):
        return {'universe_name' : self.universe_name, 
                'building_block_classes' : [i.dump() for i in self.building_block_classes]}
        
    def match(self, mol: Chem.Mol) -> dict:
        return {k: v for d in [i.match(mol) for i in self.building_block_classes] for k, v in d.items()}

In [ ]:
rxn_universe = ReactionUniverse('all_rxns', BB_CLASSES.values())
match_names = list(rxn_universe.match(Chem.MolFromSmiles('CC(NC1CCN(C(=O)OC(C)(C)C)CC1)c1cccc(Br)c1')).keys())
assert match_names == ['SecondaryAmines', 'Bifunctional', 'ArylHalide']

In [ ]:
#| export
class BuildingBlock(Molecule):
    def __init__(self, 
                 smile: str, 
                 data:  Optional[dict]=None):
        super().__init__(smile, data)
        
    def match_schema(self, schema: BuildingBlockSchema) -> bool:
        return schema.match(self.mol)
    
    def match_class(self, bb_class: BuildingBlockClass) -> dict:
        return bb_class.match(self.mol)
    
    def match_universe(self, rxn_universe: ReactionUniverse) -> dict:
        return rxn_universe.match(self.mol)
    
    def classify(self, rxn_universe: ReactionUniverse) -> dict:
        matches = self.match_universe(rxn_universe)
        self.add_data({rxn_universe.universe_name : matches})
        return matches

In [ ]:
bb = BuildingBlock('CC(NC1CCN(C(=O)OC(C)(C)C)CC1)c1cccc(Br)c1')
_ = bb.classify(rxn_universe)
assert list(bb.data[rxn_universe.universe_name].keys()) == ['SecondaryAmines', 'Bifunctional', 'ArylHalide']

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()